<a href="https://colab.research.google.com/github/Andrea91s/immobiliare_scraping_project/blob/main/Immobiliare_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import csv
import re
import itertools
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#connect to the website and fetch information
def connect(web_addr):
    resp = requests.get(web_addr)
    return BeautifulSoup(resp.content, "html.parser")

In [ ]:
# scraping raw html to find districts in the city
def get_areas(website):
    soup = connect(website)
    areas=[]
    for link_quart in soup.find_all('a', class_='in-breadcrumb__dropdownLink'):
        for i in range(len(link_quart.text.split(','))):
          areas.append(link_quart.text.split(',')[i])
    areas = [stringa.strip() for stringa in areas]
    urls = []

    for area in areas:
        url = website + area.replace(' ','-').lower()
        urls.append(url)

    return areas, urls

In [ ]:
#filtering districts to remove the unwanted ones (this should become a parameter)
def filtering_districts(filtered_areas, filtered_districts):
  lower_filtered =[]
  for word in filtered_areas:
      lower_filtered.append("-".join(word.lower().split()))

  filtered_url=[]
  for element in url:
    for word in lower_filtered:
      if word in element:
        filtered_url.append(element)
  return filtered_url

In [ ]:
#selecting all the
def select_links(filtered_url):
  announces_links = []
  for url_page in tqdm(filtered_url):
    soup = connect(url_page)
    href=soup.find_all('a', class_='in-pagination__item')
    max_pages=(int(href[-1]['href'].split('pag=')[1]))
    for page in range(max_pages):
      new_url_page=url_page + "/?pag=" + str(page+1)
      print('Scraping',new_url_page)
      soup_page=connect(new_url_page)
      all_apartments=[a['href'] for a in soup_page.find_all('a', class_='in-card__title')]
      announces_links.append(all_apartments)
  announces_links=list(itertools.chain.from_iterable(announces_links))
  return announces_links

In [ ]:
website = "https://www.immobiliare.it/vendita-case/torino/"
areas,url=get_areas(website)

filtered_areas=['Cit Turin', 'Centro', 'Santa Rita', 'Crocetta', 'San Secondo','Cenisia', 'San Salvario']

filtered_districts=[elemento for elemento in areas if elemento in filtered_areas]
filtered_url=filtering_districts(filtered_areas, filtered_districts)
print("Those are the selected districts \n")

print(filtered_url)

announces_links = select_links(filtered_url)



Those are the selected districts 

['https://www.immobiliare.it/vendita-case/torino/cit-turin', 'https://www.immobiliare.it/vendita-case/torino/santa-rita', 'https://www.immobiliare.it/vendita-case/torino/crocetta', 'https://www.immobiliare.it/vendita-case/torino/san-secondo', 'https://www.immobiliare.it/vendita-case/torino/cenisia', 'https://www.immobiliare.it/vendita-case/torino/san-salvario', 'https://www.immobiliare.it/vendita-case/torino/centro']


  0%|          | 0/7 [00:00<?, ?it/s]

Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=1
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=2
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=3
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=4
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=5
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=6
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=7
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=1
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=2
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=3
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=4
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=5
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=6
Scraping https://www.immobiliare.it/vendita-c

In [ ]:
df_scrape=pd.DataFrame()

for url_page in tqdm(announces_links):
    data = connect(url_page)
    info = data.find_all('dl', class_='in-realEstateFeatures__list')
    comp_info = pd.DataFrame()
    cleaned_id_text = []
    cleaned_id__attrb_text = []
    for n in range(len(info)):
        for i in info[n].find_all('dt'):
            cleaned_id_text.append(i.text)
        for i in info[n].find_all('dd'):
            cleaned_id__attrb_text.append(i.text)

    comp_info['Id'] = cleaned_id_text
    comp_info['Attribute'] = cleaned_id__attrb_text
    temp_df = pd.DataFrame(columns=comp_info['Id'].values)
    temp_df.loc[len(temp_df), :] = comp_info['Attribute'].values[0:len(comp_info['Id'].values)]
    temp_df['url']=url_page
    try:
      temp_df['city']=data.find_all('span', class_="in-location")[0].text
      temp_df['districts']=data.find_all('span', class_="in-location")[1].text
      temp_df['address']=data.find_all('span', class_="in-location")[2].text
    except:
      temp_df['city']='Torino'
      temp_df['districts']=''
      temp_df['address']=''

    if temp_df.columns.duplicated().any():
      temp_df=temp_df.loc[:,~temp_df.columns.duplicated()]
    df_scrape = df_scrape.append(temp_df, sort=False)
df_scrape


  0%|          | 0/1670 [00:00<?, ?it/s]

,Riferimento e Data annuncio,contratto,tipologia,superficie,locali,piano,totale piani edificio,disponibilità,altre caratteristiche,prezzo,...,Deposito cauzionale,Rialzo minimo in caso di gara,luogo vendita,Luogo presentazione,termine presentazione,registro,Rito,Deposito conto spese,unità,Data di inizio lavori e di consegna prevista
0,Vittorio117 - 11/08/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,117 m² - Vedi dettaglio,"3 (2 camere da letto, 1 altro), 1 bagno, cucin...","1°, con ascensore, con accesso disabili",7 piani,Libero,Cancello elettricoMansardaFibra otticaArmadio ...,€ 230.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Vittorio45 - 09/08/2023,Vendita,Appartamento | Classe immobile signorile,45 m² - Vedi dettaglio,"1 camera da letto, 1 bagno, cucina abitabile","1°, con ascensore",7 piani,Libero,Cancello elettricoMansardaFibra otticaPorta bl...,€ 88.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Vittorio210162mq - 09/08/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,162 m² - Vedi dettaglio,"4 (2 camere da letto, 2 altri), 2 bagni, cucin...","1°, con ascensore",7 piani,Libero,Cancello elettricoMansardaFibra otticaArmadio ...,€ 318.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,EK-105222703 - 05/08/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,68 m² - Vedi dettaglio,"3 (1 camera da letto, 2 altri), 1 bagno, cucin...","3°, con ascensore",4 piani,Libero,BalconeImpianto tv centralizzatoCantinaInfissi...,€ 238.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,EK-105148745 - 07/08/2023,Vendita,Loft | Nuda proprietà,150 m² - Vedi dettaglio,"3 (1 camera da letto, 2 altri), 2 bagni",Piano terra,1 piano,NaN,CucinaParzialmente ArredatoInfissi esterni in ...,€ 338.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,60749366 - 20/05/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,155 m²,"5 (2 camere da letto, 3 altri), 1 bagno, cucin...","1° piano, con ascensore, con accesso disabili",2 piani,Libero,TerrazzaPortiere intera giornata,€ 511.500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60750397 - 19/05/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,116 m²,"2 (1 camera da letto, 1 altro), 1 bagno, cucin...",2° piano,2 piani,Libero,Portiere intera giornata,€ 382.800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"Via Lagrange 7, Torino - 19/05/2023",Vendita | Immobile a reddito,Appartamento | Intera proprietà | Immobile di ...,140 m² - Vedi dettaglio,"5 (3 camere da letto, 2 altri), 2 bagni, cucin...","1°, con ascensore, con accesso disabili",6 piani,Libero,Cancello elettricoFibra otticaVideoCitofonoImp...,€ 630.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ver35 - 11/07/2023,Vendita | Immobile a reddito,Appartamento | Intera proprietà | Classe immob...,149 m² - Vedi dettaglio,"5 (4 camere da letto, 1 altro), 1 bagno, cucin...",1°,3 piani,Libero,Fibra otticaBalconeCantinaInfissi esterni in v...,€ 423.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from google.colab import drive
drive.mount('drive')
df_scrape.to_csv('/content/drive/MyDrive/Colab Notebooks/immobiliare_df.csv')


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
